# 🎓 AI Security Education: Notebook 9
## Real-time Monitoring Dashboard

**Duration**: 90 minutes  
**Difficulty**: 🔴 Advanced  
**Prerequisites**: Completed Notebooks 1-8

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Build interactive security dashboards with Streamlit
- ✅ Implement real-time attack detection
- ✅ Create automated alert systems
- ✅ Integrate with SIEM platforms
- ✅ Deploy production monitoring

---

## 📊 Why Real-time Monitoring?

### From Testing to Operations

**Notebooks 1-8**: You learned to attack, defend, and test  
**Notebook 9**: Now monitor and respond in real-time!

### The Monitoring Gap

Most AI security failures happen because:
- ❌ Attacks detected too late (hours/days after)
- ❌ No visibility into attack patterns
- ❌ Manual log review (slow, error-prone)
- ❌ No automated response

**Real-time monitoring solves all of these!**

### 🇦🇺 Australian Compliance

**ACSC Essential Eight**:
- Application control → Monitor AI inputs
- Security monitoring → Required for all systems

**Privacy Act 1988**:
- APP 11: Security safeguards include monitoring
- Notifiable Data Breaches: Must detect within 30 days

**Real-time monitoring is not optional - it's required!**

---

## 🏗️ Section 1: Architecture Overview

### Monitoring System Components

```
┌─────────────────────────────────────────────────────────┐
│                   USER REQUESTS                         │
└────────────────────┬────────────────────────────────────┘
                     │
                     ↓
          ┌──────────────────────┐
          │   INPUT VALIDATOR    │
          │   (Layer 1)          │
          └──────────┬───────────┘
                     │
                     ↓
          ┌──────────────────────┐
          │   AI MODEL           │
          └──────────┬───────────┘
                     │
                     ↓
          ┌──────────────────────┐
          │   OUTPUT FILTER      │
          │   (Layer 4)          │
          └──────────┬───────────┘
                     │
     ┌───────────────┼───────────────┐
     │               │               │
     ↓               ↓               ↓
┌─────────┐  ┌─────────────┐  ┌──────────┐
│ LOGGER  │  │  DASHBOARD  │  │  ALERTS  │
│ (Layer5)│  │  (Streamlit)│  │  (Email) │
└────┬────┘  └──────┬──────┘  └────┬─────┘
     │              │              │
     └──────────────┴──────────────┘
                    │
                    ↓
           ┌────────────────┐
           │  SIEM/Splunk   │
           └────────────────┘
```

---

In [ ]:
# Prerequisites check
import sys
print("🔍 PREREQUISITES CHECK")
print("=" * 80)
print(f"Python: {sys.version}")
print("\n✅ This notebook runs on CPU - no GPU required for monitoring!")
print("=" * 80)

In [ ]:
# Install required packages
!pip install -q streamlit plotly pandas numpy altair watchdog

In [ ]:
import json
import time
from datetime import datetime, timedelta
from typing import List, Dict
from dataclasses import dataclass, field
from collections import defaultdict, deque
import random

import pandas as pd
import numpy as np

print("✅ Libraries imported!")

## 💾 Section 2: Core Monitoring System

Build the backend monitoring infrastructure.

---

In [ ]:
@dataclass
class SecurityEvent:
    """A security event to be monitored"""
    timestamp: str
    event_type: str  # 'attack_detected', 'attack_blocked', 'normal_request'
    user_id: str
    ip_address: str
    attack_category: str  # 'DAN', 'Encoding', 'Injection', etc.
    severity: str  # 'low', 'medium', 'high', 'critical'
    blocked: bool
    details: Dict = field(default_factory=dict)


class RealTimeMonitor:
    """Real-time security monitoring system"""
    
    def __init__(self, max_history=1000):
        self.events = deque(maxlen=max_history)
        self.stats = {
            'total_requests': 0,
            'attacks_detected': 0,
            'attacks_blocked': 0,
            'false_positives': 0
        }
        self.ip_tracking = defaultdict(list)
        self.alert_thresholds = {
            'attacks_per_minute': 10,
            'attacks_per_ip': 5,
            'critical_attacks': 1
        }
    
    def log_event(self, event: SecurityEvent):
        """Log a security event"""
        self.events.append(event)
        self.stats['total_requests'] += 1
        
        if event.event_type == 'attack_detected':
            self.stats['attacks_detected'] += 1
            if event.blocked:
                self.stats['attacks_blocked'] += 1
        
        # Track IP
        self.ip_tracking[event.ip_address].append(event.timestamp)
        
        # Check if alert needed
        return self.check_alert_conditions(event)
    
    def check_alert_conditions(self, event: SecurityEvent) -> List[str]:
        """Check if alerts should be triggered"""
        alerts = []
        
        # Critical severity
        if event.severity == 'critical' and not event.blocked:
            alerts.append(f"CRITICAL: Unblocked {event.attack_category} attack from {event.ip_address}")
        
        # IP-based threshold
        recent_from_ip = [e for e in self.events if e.ip_address == event.ip_address and e.event_type == 'attack_detected']
        if len(recent_from_ip) >= self.alert_thresholds['attacks_per_ip']:
            alerts.append(f"WARNING: {len(recent_from_ip)} attacks from IP {event.ip_address}")
        
        # Time-based threshold  
        one_min_ago = (datetime.now() - timedelta(minutes=1)).isoformat()
        recent_attacks = [e for e in self.events if e.timestamp > one_min_ago and e.event_type == 'attack_detected']
        if len(recent_attacks) >= self.alert_thresholds['attacks_per_minute']:
            alerts.append(f"WARNING: {len(recent_attacks)} attacks in the last minute")
        
        return alerts
    
    def get_recent_events(self, minutes=5) -> List[SecurityEvent]:
        """Get events from last N minutes"""
        cutoff = (datetime.now() - timedelta(minutes=minutes)).isoformat()
        return [e for e in self.events if e.timestamp > cutoff]
    
    def get_statistics(self) -> Dict:
        """Get current statistics"""
        return {
            **self.stats,
            'block_rate': self.stats['attacks_blocked'] / max(self.stats['attacks_detected'], 1) * 100,
            'attack_rate': self.stats['attacks_detected'] / max(self.stats['total_requests'], 1) * 100
        }


# Initialize monitor
monitor = RealTimeMonitor()
print("✅ Real-time monitor initialized!")

In [ ]:
# Simulate some security events for testing
def simulate_events(count=50):
    """Generate simulated security events"""
    attack_types = ['DAN', 'Encoding', 'Injection', 'Skeleton_Key', 'Extraction']
    severities = ['low', 'medium', 'high', 'critical']
    ips = [f"192.168.1.{i}" for i in range(1, 20)]
    
    for i in range(count):
        is_attack = random.random() < 0.3  # 30% attacks
        
        event = SecurityEvent(
            timestamp=(datetime.now() - timedelta(minutes=random.randint(0, 60))).isoformat(),
            event_type='attack_detected' if is_attack else 'normal_request',
            user_id=f"user{random.randint(1, 100)}",
            ip_address=random.choice(ips),
            attack_category=random.choice(attack_types) if is_attack else 'none',
            severity=random.choice(severities) if is_attack else 'low',
            blocked=random.random() < 0.85 if is_attack else True,  # 85% block rate
            details={}
        )
        
        alerts = monitor.log_event(event)
        if alerts:
            for alert in alerts:
                print(f"🚨 {alert}")

# Generate test data
print("📊 Generating simulated security events...")
simulate_events(100)
print(f"\n✅ Generated {len(monitor.events)} events")
print(f"\nStatistics:")
for key, value in monitor.get_statistics().items():
    print(f"  {key}: {value if isinstance(value, int) else f'{value:.1f}'}")

## 📊 Section 3: Streamlit Dashboard Code

Complete interactive dashboard implementation.

**Save this as `security_dashboard.py` and run with:** `streamlit run security_dashboard.py`

---

In [ ]:
# Save complete Streamlit dashboard code to file
dashboard_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import time

st.set_page_config(
    page_title="AI Security Monitor",
    page_icon="🛡️",
    layout="wide"
)

# Title
st.title("🛡️ AI Security Real-time Monitor")
st.markdown("**Australian AI Security Dashboard** | Privacy Act 1988 Compliant")

# Sidebar
st.sidebar.header("⚙️ Configuration")
refresh_rate = st.sidebar.slider("Refresh Rate (seconds)", 1, 60, 5)
time_window = st.sidebar.selectbox("Time Window", ["Last 5 min", "Last 15 min", "Last 1 hour", "Last 24 hours"])

# Metrics
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric("Total Requests", "1,234", "+12")

with col2:
    st.metric("Attacks Detected", "45", "+3", delta_color="inverse")

with col3:
    st.metric("Attacks Blocked", "42", "+3")

with col4:
    st.metric("Block Rate", "93.3%", "+2.1%")

# Main dashboard
tab1, tab2, tab3, tab4 = st.tabs(["📊 Overview", "🚨 Alerts", "📈 Analytics", "⚙️ Settings"])

with tab1:
    st.subheader("Attack Detection Timeline")
    
    # Sample data
    timeline_data = pd.DataFrame({
        'Time': pd.date_range(start=datetime.now()-timedelta(hours=1), periods=60, freq='1min'),
        'Attacks': [random.randint(0, 5) for _ in range(60)],
        'Blocked': [random.randint(0, 5) for _ in range(60)]
    })
    
    fig = px.line(timeline_data, x='Time', y=['Attacks', 'Blocked'], 
                  title='Attack Detection Over Time')
    st.plotly_chart(fig, use_container_width=True)
    
    # Attack category breakdown
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Attack Categories")
        category_data = pd.DataFrame({
            'Category': ['DAN', 'Encoding', 'Injection', 'Skeleton Key', 'Other'],
            'Count': [15, 12, 8, 6, 4]
        })
        fig2 = px.pie(category_data, values='Count', names='Category')
        st.plotly_chart(fig2, use_container_width=True)
    
    with col2:
        st.subheader("Severity Distribution")
        severity_data = pd.DataFrame({
            'Severity': ['Critical', 'High', 'Medium', 'Low'],
            'Count': [3, 12, 18, 12]
        })
        fig3 = px.bar(severity_data, x='Severity', y='Count', 
                     color='Severity',
                     color_discrete_map={'Critical': 'red', 'High': 'orange', 
                                       'Medium': 'yellow', 'Low': 'green'})
        st.plotly_chart(fig3, use_container_width=True)

with tab2:
    st.subheader("🚨 Active Alerts")
    
    # Alert table
    alerts = pd.DataFrame({
        'Time': [datetime.now()-timedelta(minutes=i) for i in [2, 15, 30]],
        'Severity': ['🔴 Critical', '🟠 High', '🟡 Medium'],
        'Message': [
            'Multiple DAN attacks from 192.168.1.100',
            'Skeleton Key attempt detected',
            'Unusual encoding pattern observed'
        ],
        'Status': ['Active', 'Investigating', 'Resolved']
    })
    
    st.dataframe(alerts, use_container_width=True, hide_index=True)
    
    if st.button("🔔 Send Test Alert"):
        st.success("Test alert sent to admin@example.com")

with tab3:
    st.subheader("📈 Security Analytics")
    
    # Top attacking IPs
    ip_data = pd.DataFrame({
        'IP Address': [f'192.168.1.{i}' for i in [100, 101, 102, 103, 104]],
        'Attack Count': [25, 18, 15, 12, 10],
        'Country': ['🇦🇺 Australia', '🇺🇸 USA', '🇨🇳 China', '🇷🇺 Russia', '🇧🇷 Brazil']
    })
    
    st.dataframe(ip_data, use_container_width=True, hide_index=True)
    
    # Attack success rate over time
    success_data = pd.DataFrame({
        'Date': pd.date_range(start=datetime.now()-timedelta(days=7), periods=7, freq='1D'),
        'Success Rate': [5.2, 4.8, 6.1, 4.5, 3.9, 3.2, 2.8]
    })
    
    fig4 = px.line(success_data, x='Date', y='Success Rate',
                   title='Attack Success Rate Trend (Lower is Better)')
    st.plotly_chart(fig4, use_container_width=True)

with tab4:
    st.subheader("⚙️ Alert Configuration")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.number_input("Attacks per minute threshold", value=10, min_value=1)
        st.number_input("Attacks per IP threshold", value=5, min_value=1)
        st.checkbox("Enable email alerts", value=True)
        st.checkbox("Enable Slack notifications", value=False)
    
    with col2:
        st.text_input("Alert email", "security@example.com")
        st.text_input("Slack webhook URL", "")
        st.selectbox("Alert priority", ["All", "High and Critical only", "Critical only"])
    
    if st.button("💾 Save Configuration"):
        st.success("Configuration saved successfully!")

# Footer
st.markdown("---")
st.markdown("🇦🇺 **Australian Compliance**: Privacy Act 1988 | ACSC Essential Eight | OAIC Guidelines")
st.markdown("Last updated: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S AEST"))

# Auto-refresh
time.sleep(refresh_rate)
st.rerun()
'''

# Save to file
with open('security_dashboard.py', 'w') as f:
    f.write(dashboard_code)

print("✅ Dashboard code saved to 'security_dashboard.py'")
print("\n📝 To run the dashboard:")
print("   streamlit run security_dashboard.py")
print("\n💡 The dashboard will be available at http://localhost:8501")

## 🚨 Section 4: Alert System

Automated alerting when thresholds are exceeded.

---

In [ ]:
class AlertSystem:
    """Automated alert system"""
    
    def __init__(self):
        self.alert_log = []
        self.email_enabled = True
        self.slack_enabled = False
    
    def send_alert(self, severity: str, message: str, details: Dict = None):
        """Send an alert through configured channels"""
        alert = {
            'timestamp': datetime.now().isoformat(),
            'severity': severity,
            'message': message,
            'details': details or {}
        }
        
        self.alert_log.append(alert)
        
        # Email alert
        if self.email_enabled:
            self._send_email(alert)
        
        # Slack alert
        if self.slack_enabled:
            self._send_slack(alert)
        
        return alert
    
    def _send_email(self, alert: Dict):
        """Send email alert (simulated)"""
        print(f"📧 Email Alert Sent:")
        print(f"   To: security@example.com")
        print(f"   Subject: [{alert['severity'].upper()}] Security Alert")
        print(f"   Body: {alert['message']}")
    
    def _send_slack(self, alert: Dict):
        """Send Slack alert (simulated)"""
        print(f"💬 Slack Alert Sent: {alert['message']}")
    
    def get_recent_alerts(self, minutes=60) -> List[Dict]:
        """Get alerts from last N minutes"""
        cutoff = (datetime.now() - timedelta(minutes=minutes)).isoformat()
        return [a for a in self.alert_log if a['timestamp'] > cutoff]


# Initialize alert system
alert_system = AlertSystem()

# Test alerts
print("🧪 Testing alert system...\n")
alert_system.send_alert('critical', 'Multiple DAN attacks from 192.168.1.100')
alert_system.send_alert('high', 'Skeleton Key attempt detected', {'ip': '192.168.1.101'})
alert_system.send_alert('medium', 'Unusual encoding pattern', {'category': 'Base64'})

print(f"\n✅ Alert system operational! {len(alert_system.alert_log)} alerts logged.")

## 🔗 Section 5: SIEM Integration

Integrate with Security Information and Event Management platforms.

---

In [ ]:
class SIEMIntegration:
    """SIEM platform integration"""
    
    def __init__(self, platform='splunk'):
        self.platform = platform
        self.connected = False
    
    def connect(self, host: str, port: int, api_key: str):
        """Connect to SIEM platform"""
        print(f"🔗 Connecting to {self.platform.upper()} at {host}:{port}...")
        # In production: actual connection code
        self.connected = True
        print(f"✅ Connected to {self.platform.upper()}")
    
    def send_event(self, event: SecurityEvent):
        """Send event to SIEM"""
        if not self.connected:
            print("⚠️  Not connected to SIEM")
            return
        
        # Format for SIEM (example: Splunk HEC format)
        siem_event = {
            'time': event.timestamp,
            'event': {
                'type': event.event_type,
                'category': event.attack_category,
                'severity': event.severity,
                'user': event.user_id,
                'ip': event.ip_address,
                'blocked': event.blocked
            },
            'sourcetype': 'ai_security',
            'index': 'security'
        }
        
        # In production: send via HTTP Event Collector or similar
        print(f"📤 Sent to {self.platform.upper()}: {event.event_type}")
        return siem_event


# Example SIEM integration
print("🔧 SIEM Integration Examples:\n")

# Splunk
splunk = SIEMIntegration('splunk')
splunk.connect('splunk.example.com', 8088, 'your-api-key')

# Send test event
test_event = SecurityEvent(
    timestamp=datetime.now().isoformat(),
    event_type='attack_detected',
    user_id='test_user',
    ip_address='192.168.1.100',
    attack_category='DAN',
    severity='high',
    blocked=True
)
splunk.send_event(test_event)

print("\n✅ SIEM integration ready!")

## 🚀 Section 6: Production Deployment

Deploy monitoring in production environments.

---

### Production Deployment Checklist

#### 1. Docker Containerization

```dockerfile
# Dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY security_dashboard.py .

EXPOSE 8501

CMD ["streamlit", "run", "security_dashboard.py", "--server.port=8501", "--server.address=0.0.0.0"]
```

#### 2. Kubernetes Deployment

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: security-dashboard
spec:
  replicas: 2
  selector:
    matchLabels:
      app: security-dashboard
  template:
    metadata:
      labels:
        app: security-dashboard
    spec:
      containers:
      - name: dashboard
        image: your-registry/security-dashboard:latest
        ports:
        - containerPort: 8501
```

#### 3. Australian Data Residency

```python
# Ensure data stays in Australia (Privacy Act 1988)
AWS_REGION = 'ap-southeast-2'  # Sydney
AZURE_REGION = 'australiaeast'  # Sydney
```

#### 4. Monitoring the Monitor

```python
# Health check endpoint
@app.route('/health')
def health_check():
    return {'status': 'healthy', 'uptime': get_uptime()}
```

---

## 🎓 Section 7: Assessment & Resources

---

### 📝 Assessment Quiz

**Question 1**: Why is real-time monitoring required under Privacy Act 1988?
- A) It's not required
- B) APP 11 requires security safeguards including monitoring ✅ CORRECT
- C) Only for government agencies
- D) Optional best practice

**Question 2**: What's the recommended alert threshold for attacks per IP?
- A) 1-2 attacks
- B) 5-10 attacks ✅ CORRECT (balance between security and usability)
- C) 50+ attacks
- D) No threshold needed

**Question 3**: Which ACSC Essential Eight control relates to monitoring?
- A) Application control
- B) Security monitoring ✅ CORRECT
- C) Backups
- D) User training

**Question 4**: How often should the dashboard refresh in production?
- A) Every second
- B) Every 5-30 seconds ✅ CORRECT (balance between real-time and performance)
- C) Every hour
- D) Manual refresh only

---

### 🎯 Key Takeaways

**What You Learned:**
1. ✅ **Real-time monitoring is mandatory** - Not optional for Australian compliance
2. ✅ **Complete monitoring system** - Events, alerts, dashboards, SIEM
3. ✅ **Streamlit dashboards** - Production-ready interactive monitoring
4. ✅ **Alert automation** - Email, Slack, threshold-based
5. ✅ **SIEM integration** - Splunk, ELK, enterprise platforms

**Australian Compliance:**
- ✅ Privacy Act 1988 (APP 11)
- ✅ ACSC Essential Eight (Security monitoring)
- ✅ Notifiable Data Breaches (30-day detection)
- ✅ Data residency (Australian servers)

---

### 💡 Further Reading

- **Streamlit Documentation**: https://docs.streamlit.io/
- **Plotly Charts**: https://plotly.com/python/
- **ACSC Security Monitoring**: https://www.cyber.gov.au/
- **Splunk Integration**: https://docs.splunk.com/
- **Privacy Act 1988**: https://www.oaic.gov.au/

---

### 🚀 What's Next?

**In Notebook 10: CTF Security Challenges**, you'll:
- Solve 15 progressive security challenges
- Test all skills learned (Notebooks 1-9)
- Compete for high scores
- Earn certification

**Put your skills to the test!** 🏆

---

## 🎉 Congratulations!

You can now build production monitoring systems!

**Skills Acquired:**
- ✅ Real-time security monitoring
- ✅ Interactive Streamlit dashboards
- ✅ Automated alert systems
- ✅ SIEM platform integration
- ✅ Production deployment

**You're ready for Notebook 10!** 🚀

---